# Estudantes Estrangeiros na Cidade de São Paulo

Neste tutorial, realizamos operações básicas para explorar a base de microdados de matrículas e filtrar uma das informações mais demandadas via Lei de Acesso à Informação: alunos estrangeiros, por nacionalidade, modalidade de ensino, idade etc. 

[Saiba mais](https://github.com/prefeiturasp/dados-educacao/blob/master/README.md) sobre os tutoriais dos dados abertos da SME.

Importante: também estamos aprendendo! Tem sugestões para melhorar os códigos? Não deixe de contribuir com este repositório :)

## Passo 1: Importando os dados

In [1]:
import pandas as pd
import numpy as np

Importar o arquivo .csv disponível no Portal de Dados Abertos, já descompactado:

In [35]:
df_2016 = pd.read_csv('Microdados_EOL_Matriculas_2016.csv', sep=';', encoding='ISO-8859-1', low_memory=False)

## Passo 2: Conhecendo os dados

In [4]:
df_2016.head()

#Visualiza os 5 primeiros registros da base de dados.

,AN_LETIVO,CD_UNIDADE_EDUCACAO,NOME_DISTRITO,CD_SETOR,TIPO_ESCOLA,NOME_ESCOLA,DRE,CD_INEP_ESCOLA,SITUACAO_ESCOLA,CD_TURMA,...,SIT_AL_TRANSF,SIT_AL_FALEC,SIT_AL_NCOM,SIT_AL_EXCL,SIT_AL_CES,SIT_AL_REMAN,SIT_AL_RECLAS,SIT_AL_ATIVO,CD_PARECER_CONCL_FIN,DESC_PARECER_CONCL_FIN
0,2016,306329,VILA GUILHERME,8603,CR.P.CONV,"LUIZ BIASI, CON",DIRETORIA REGIONAL DE EDUCACAO JACANA/TREMEMBE,35183362.0,ATIVA,1670560,...,0,0,0,0,0,0,0,0,NaN,NaN
1,2016,306310,SACOMA,6802,CR.P.CONV,NOSSA SENHORA DAS MERCES,DIRETORIA REGIONAL DE EDUCACAO IPIRANGA,35181183.0,ATIVA,1670600,...,0,0,0,0,0,0,0,0,NaN,NaN
2,2016,306310,SACOMA,6802,CR.P.CONV,NOSSA SENHORA DAS MERCES,DIRETORIA REGIONAL DE EDUCACAO IPIRANGA,35181183.0,ATIVA,1670603,...,0,0,0,0,0,0,0,0,NaN,NaN
3,2016,400564,PIRITUBA,6301,CEI DIRET,RAQUEL ZUMBANO ALTMAN,DIRETORIA REGIONAL DE EDUCACAO PIRITUBA/JARAGUA,35129938.0,ATIVA,1670205,...,0,0,0,0,0,0,0,0,NaN,NaN
4,2016,400686,GRAJAU,3001,CEI INDIR,PARQUE AMERICA,DIRETORIA REGIONAL DE EDUCACAO CAPELA DO SOCORRO,35387332.0,ATIVA,1670268,...,0,0,0,0,0,0,0,0,NaN,NaN


In [36]:
df_2016.columns
#Visualiza o nome de cada coluna disponível

Index(['AN_LETIVO', 'CD_UNIDADE_EDUCACAO', 'NOME_DISTRITO', 'CD_SETOR',
       'TIPO_ESCOLA', 'NOME_ESCOLA', 'DRE', 'CD_INEP_ESCOLA',
       'SITUACAO_ESCOLA', 'CD_TURMA', 'CD_TURNO', 'DESC_TURNO', 'CD_SERIE',
       'DESC_SERIE', 'MODALIDADE', 'MODALIDADE_SEGMENTO', 'TURMA_ESCOL',
       'NOME_TURMA', 'HORA_IN_TURMA', 'HORA_FIM_TURMA',
       'DESC_PERIODICIDADE_TURMA', 'CD_ETAPA_ENSINO', 'DESC_ETAPA_ENSINO',
       'CD_CICLO_ENSINO', 'DESC_CICLO_ENSINO', 'CD_TIPO_TURMA',
       'DESC_TIPO_TURMA', 'CD_TIPO_PROGRAMA', 'DESC_TIPO_PROGRAMA',
       'DUR_DIA_TURMA', 'DUR_DIA_HORA', 'DUR_DIA_MIN', 'DUR_SEMAN_TURMA',
       'DUR_SEM_HORA', 'DUR_SEM_MIN', 'QTD_DIAS_SEMAN_TURMA', 'X_SEMANA',
       'SEG', 'TER', 'QUA', 'QUI', 'SEX', 'SAB', 'DOM', 'CD_ALUNO_SME',
       'CD_INEP_ALUNO', 'ANO_NASC_ALUNO', 'MES_NASC_ALUNO',
       'IDADE_ALUNO_ANO_CIVIL', 'idade_aluno_31_03', 'CD_SEXO', 'CD_RACA_COR',
       'DESC_RACA_COR', 'CD_PAIS_NASC', 'DESC_PAIS_NASC', 'CD_MUN_NASC',
       'DESC_MUN_NASC'

## Passo 3: Filtrando as colunas desejadas

A partir da lista de colunas acima, vamos selecionar algumas colunas interessantes para a análise, tais como raça/cor, sexo, tipo de escola, distrito etc. Lembrando que o [dicionário de váriáveis](http://dados.prefeitura.sp.gov.br/dataset/66218af2-c669-49bc-842c-df8fcc6364ca/resource/a59b705b-febc-4968-b406-360d4a163747/download/dicionariovariaveismicrodadosmatriculasago2017.xlsx) está disponível no Portal de Dados Abertos. 

In [39]:
nacionalidades = df_2016[['AN_LETIVO', 'CD_ALUNO_SME', 'DESC_RACA_COR', 'CD_SEXO', 'TIPO_ESCOLA', 'DRE', 'NOME_DISTRITO', 'TURMA_ESCOL', 'MODALIDADE_SEGMENTO', 'DESC_SERIE', 'DESC_PAIS_NASC']]

Aqui, vamos filtrar apenas os estudantes estrangeiros matriculados na rede. Vamos aplicar dois filtros: na coluna que descreve a nacionalidade, vamos selecionar tudo o que é diferente de "Brasil". Também vamos adicionar um filtro na coluna "Turma_Escol", que separa o que é turma de atividades complementares das turmas regulares.

In [40]:
estrangeiros = nacionalidades[(nacionalidades['DESC_PAIS_NASC'] != 'BRASIL') & (nacionalidades['TURMA_ESCOL'] == 'S')]

In [42]:
estrangeiros.head()
#Visualiza como ficou!

,AN_LETIVO,CD_ALUNO_SME,DESC_RACA_COR,CD_SEXO,TIPO_ESCOLA,DRE,NOME_DISTRITO,TURMA_ESCOL,MODALIDADE_SEGMENTO,DESC_SERIE,DESC_PAIS_NASC
11,2016,6261502,NAO DECLARADA,F,CR.P.CONV,DIRETORIA REGIONAL DE EDUCACAO BUTANTA,BUTANTA,S,CRECHE,MINI GRUPO II,INDIA
94,2016,6448463,NAO DECLARADA,F,CR.P.CONV,DIRETORIA REGIONAL DE EDUCACAO PIRITUBA/JARAGUA,PIRITUBA,S,CRECHE,BERCARIO I,BOLIVIA
125,2016,6417892,NAO DECLARADA,M,EMEF,DIRETORIA REGIONAL DE EDUCACAO CAMPO LIMPO,JARDIM SAO LUIS,S,Fund2,C.II 6.ANO F9,NaN
219,2016,6051793,NAO DECLARADA,F,CEU EMEF,DIRETORIA REGIONAL DE EDUCACAO PIRITUBA/JARAGUA,SAO DOMINGOS,S,EJA5A8,EJA COMPL II,NaN
242,2016,6296484,NAO DECLARADA,M,EMEF,DIRETORIA REGIONAL DE EDUCACAO BUTANTA,RIO PEQUENO,S,EJA5A8,EJA FINAL I,NaN


## Passo 4: Limpando a base

In [45]:
estrangeiros.shape

(15153, 11)

O código acima indica que a base resultante tem 15.153 registros. Mas será que isso significa que há 15.153 estrangeiros? Repare que na coluna de Nacionalidade há alguns valores que aparecem como "Nan". Isso significa que a Nacionalidade não foi cadastrada na escola, por qualquer motivo. É importante saber dessa condição, mas vamos limpar a base para seguir a análise com os dados disponíveis, excluindo as linhas que possuem "Nan":

In [47]:
estrangeiros_decl = estrangeiros.dropna(how='any', subset=['DESC_PAIS_NASC'])

In [50]:
estrangeiros_decl.shape

(6630, 11)

De fato, apenas 6.630 estudantes que não estão cadastrados com País de Nascimento "Brasil" possuem um País declarado.

## Passo 5: Algumas análises

Com a base pronta, é possível realizar análises, tais como ordenar, agrupar e fazer gráficos. Por exemplo, ordenar por modalidade/segmento de ensino:

In [51]:
estrangeiros.groupby('MODALIDADE_SEGMENTO').CD_ALUNO_SME.count()

MODALIDADE_SEGMENTO
CONVEE       1
CRECHE    1137
EDPROF      22
EJA1A4    1151
EJA5A8    4205
Fund1     3948
Fund2     2665
MEDIO       13
MOVA       164
PRE       1795
PROJ        52
Name: CD_ALUNO_SME, dtype: int64

Agrupar por nacionalidade e ordenar pelas nacionalidade mais frequentes:

In [29]:
por_nac = estrangeiros_decl.groupby('DESC_PAIS_NASC').CD_ALUNO_SME.count().sort_values(ascending = False)

In [55]:
df_nac = por_nac.to_frame()
df_nac.head(10)
#Visualiza as 10 mais frequentes

,CD_ALUNO_SME
DESC_PAIS_NASC,
BOLIVIA,3658
ANGOLA,811
REPUBLICA DO HAITI,371
JAPAO,285
PARAGUAI,186
PERU,181
ARGENTINA,164
CONGO,138
COLOMBIA,99


## Passo 6: Exportando para Excel ou CSV

Depois de trabalhar a base, você pode querer exportá-la para o Excel ou outro programa. Os códigos abaixo realizam essa operação:

In [58]:
estrangeiros_decl.to_csv('estrangeiros_2016.csv', index=False)

In [59]:
estrangeiros_decl.to_excel('estrangeiros_2016.xls', index=False)

Pronto! Estão salvos no mesmo diretório em que este arquivo de tutorial está. 